In [19]:
import json
import tabulate
import evaluation

# Load the JSON data
with open("metrics.json", "r") as f:
    metrics = json.load(f)

# Prepare the data for tabulation
table_data = []
headers = ["Name", "Origin", "Faulty engine", "Size(Byte)", "Function Count"]

for test_name, data in metrics.items():
    metadata = data['metadata']
    table_data.append([
        test_name,
        metadata['origin'],
        metadata['engine'],
        metadata['size'],
        metadata['function_count']
    ])

# Generate the LaTeX table
latex_table = tabulate.tabulate(table_data, headers, tablefmt="latex_raw")

# Modify the LaTeX table string
latex_table = latex_table.replace("#", "\\#")

# Save the table to a file
rq1_table_path = f'{evaluation.PAPER_PATH}/table/eval_set.tex'
with open(rq1_table_path, 'w') as file:
    file.write(latex_table)
    print(f"Table saved to {rq1_table_path}")

Table saved to /home/don/rr-reduce-paper/issta_2025/table/eval_set.tex


In [3]:
!FIDX_LIMIT=1000 python rq1.py

RQ1
len(testset): 21
RUNNING: wamr#2861 with 1 functions
FINISHED: wamr#2861 in 2.237379789352417 seconds
RUNNING: wasmedge#3018 with 12 functions
FINISHED: wasmedge#3018 in 11.197264194488525 seconds
RUNNING: wamr#2789 with 58 functions
FINISHED: wamr#2789 in 45.468804597854614 seconds
RUNNING: wamr#2450 with 66 functions
FINISHED: wamr#2450 in 23.624891996383667 seconds
RUNNING: wamr#2862 with 73 functions
FINISHED: wamr#2862 in 26.244394063949585 seconds
RUNNING: wasmedge#3019 with 76 functions
FINISHED: wasmedge#3019 in 56.48368167877197 seconds
RUNNING: wasmedge#3076 with 131 functions
FINISHED: wasmedge#3076 in 97.36351919174194 seconds
RUNNING: mandelbrot with 167 functions
FINISHED: mandelbrot in 7.61648416519165 seconds
RUNNING: guiicons with 505 functions
FINISHED: guiicons in 387.5126144886017 seconds
RUNNING: rguilayout with 510 functions
FINISHED: rguilayout in 395.0798075199127 seconds
RUNNING: rguistyler with 575 functions
FINISHED: rguistyler in 450.84219622612 seconds


In [21]:
import json
import tabulate
import evaluation

# Load the JSON data
with open("metrics.json", "r") as f:
    metrics = json.load(f)

# Prepare the data for tabulation
table_data = []
headers = ["Name", "Function Count",  "Split Success", "Record and Replay Success"]

for test_name, data in metrics.items():
    metadata = data['metadata']
    try:
        split_success = sum(1 for obj in data['rq1'].values() if obj['slice-dice'] != 'fail')
        rr_success = sum(1 for obj in data['rq1'].values() if obj['wasm-r3'] != 'fail')
        table_data.append([
            test_name,
            metadata['function_count'],
            f'{split_success}({split_success / metadata["function_count"]:.2%})',
            f'{rr_success}({rr_success / metadata["function_count"]:.2%})',
        ])
    except KeyError:
        continue

# Generate the LaTeX table
latex_table = tabulate.tabulate(table_data, headers, tablefmt="latex_raw")

# Modify the LaTeX table string
latex_table = latex_table.replace("#", "\\#")

# Save the table to a file
rq1_table_path = f'{evaluation.PAPER_PATH}/table/rq1.tex'
with open(rq1_table_path, 'w') as file:
    file.write(latex_table)
    print(latex_table)
    print(f"Table saved to {rq1_table_path}")

\begin{tabular}{lrll}
\hline
 Name          &   Function Count & Split Success   & Record and Replay Success   \\
\hline
 wamr\#2861     &                1 & 1(100.00%)      & 1(100.00%)                  \\
 wasmedge\#3018 &               12 & 12(100.00%)     & 4(33.33%)                   \\
 wamr\#2789     &               58 & 58(100.00%)     & 38(65.52%)                  \\
 wamr\#2450     &               66 & 66(100.00%)     & 61(92.42%)                  \\
 wamr\#2862     &               73 & 73(100.00%)     & 73(100.00%)                 \\
 wasmedge\#3019 &               76 & 76(100.00%)     & 70(92.11%)                  \\
 wasmedge\#3076 &              131 & 131(100.00%)    & 127(96.95%)                 \\
 mandelbrot    &              167 & 0(0.00%)        & 0(0.00%)                    \\
 guiicons      &              505 & 505(100.00%)    & 358(70.89%)                 \\
 rguilayout    &              510 & 510(100.00%)    & 353(69.22%)                 \\
 rguistyler    &      

In [6]:
!python rq2.py

['boa', 'guiicons', 'funky-kart', 'jsc', 'rfxgen', 'rguilayout', 'rguistyler', 'riconpacker', 'sqlgui', 'commanderkeen', 'hydro', 'rtexviewer', 'mandelbrot', 'wasmedge#3057', 'wasmedge#3076', 'wamr#2450', 'wasmedge#3019', 'wamr#2789', 'wamr#2862', 'wasmedge#3018', 'wamr#2861']
len(testset): 21


In [ ]:
import tabulate
import eval